In [1]:
import pandas as pd

### Getting the DF

In [25]:
df = pd.read_csv('data/twitter.csv')

In [21]:
df.head()

,canton_code,canton,Unnamed: 0,month,language,gender,time_period,avg_sentiment,count
0,AG,Aargau,11,8,en,MALE,Day,0.16,50
1,AG,Aargau,14,10,en,FEMALE,Day,0.10,10
2,AG,Aargau,38,2,de,UNKNOWN,Day,1.00,6
3,AG,Aargau,50,3,de,FEMALE,Day,1.00,6
4,AG,Aargau,104,4,de,UNKNOWN,Night,1.00,5


In [3]:
code_mapping = pd.read_csv('data/canton_code.csv').ix[:,1:3]

In [4]:
code_mapping.columns = ['canton_code', 'canton']

In [5]:
df.head()

,Unnamed: 0,Unnamed: 0.1,canton,month,language,gender,time_period,count,sentiment
0,0,0,Basel-City,7,English,Male,Day,2764,0.163169
1,1,1,Geneva,8,English,Male,Day,13086,0.136864
2,2,2,Solothurn,9,English,Female,Night,69,0.260870
3,3,3,Schwyz,5,English,Unknown,Day,6,0.500000
4,4,4,Basel-City,9,French,Unknown,Night,2,1.000000


In [32]:
new_df = code_mapping.merge(new_df, on = 'canton')

In [26]:
new_df = df.copy()

In [33]:
new_df.head()

,canton_code,canton,month,language,gender,time_period,count,sentiment
0,AG,Aargau,8,English,Male,Day,50,0.16
1,AG,Aargau,10,English,Female,Day,10,0.10
2,AG,Aargau,2,German,Unknown,Day,6,1.00
3,AG,Aargau,3,German,Female,Day,6,1.00
4,AG,Aargau,4,German,Unknown,Night,5,1.00


In [29]:
new_df.drop(["Unnamed: 0"],1, inplace=True)
# changes avg_sentiment to sentiment
new_df['sentiment'] = new_df['avg_sentiment']
new_df = new_df.drop('avg_sentiment',axis=1)

In [27]:
new_df['gender'] = new_df['gender'].map({"FEMALE":"Female","MALE":"Male", "UNKNOWN":"Unknown"})
new_df['language'] = new_df['language'].map({"de":"German","en":"English", "fr":"French"})

In [34]:
new_df.to_csv('data/twitter.csv')

### Creating the Function

In [ ]:
# function that filters out the dataframe and gives the final result wanted.
# gender, month, language

def filter_df(df, gender=True, min_month=1, max_month=10, language = True, time_period = True):
    if type(gender) is str:
        gender = (df['gender'] == gender)
    month = ((min_month <= df['month']) & (df['month'] <= max_month))
    if type(language) is str:
        language = (df['language'] == language)
    if type(time_period) is str:
        time_period = (df['time_period'] == time_period)
    filtered = df[gender & time_period & language]
    return filtered

In [ ]:
cantons_kml = ["ZH","BE","LU","UR","SZ","OW","NW","GL","ZG","FR","SO","BS","BL","SH","AR","AI","SG","GR","AG","TG","TI","VD","VS","NE","GE","JU"]
cantons_df = pd.DataFrame(cantons_kml)
cantons_df.columns = ['canton_code']


def group(filtered_df):
    filtered_df = filtered_df.groupby('canton_code').mean()
    filtered_df = filtered_df.reset_index()[['canton_code','sentiment']] # group by
    final = cantons_df.merge(filtered_df, on='canton_code', how='left')
    return final

In [ ]:
cantons_df.to_csv('cantons_kml.csv')

In [ ]:
def new_df(df, gender=True, min_month=1, max_month=10, language = True, time_period = True):
    return group(filter_df(df,gender,min_month,max_month,language,time_period))

In [ ]:
new_df(df, 'FEMALE',language='en')